In [12]:
import collections
import urllib.request
import json
import requests
import csv
import os
import shutil
import collections

In [3]:
url_tissue_rui = "https://ccf-api.hubmapconsortium.org/v1/hubmap/rui_locations.jsonld"
url_reference_organ_data = "https://hubmapconsortium.github.io/ccf-ui/rui/assets/reference-organ-data.json"
url_collision_detection = "http://192.168.1.100:8080/get-collisions"

Step 1: save all the hubmap datasets in one folder

In [8]:
def copy_csv_files_in_one_folder(input_dir, output_dir):

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for folder in os.listdir(input_dir):
        full_folder = os.path.join(input_dir, folder)
        if os.path.isdir(full_folder):
            for file_name in os.listdir(full_folder):
                if file_name.startswith('HBM'):
                    abs_file_path = os.path.join(full_folder, file_name)
                    output_file_path = os.path.join(output_dir, file_name)
                    shutil.copy(abs_file_path, output_file_path)

In [10]:
input_dir = 'C:/Users/catherine/Desktop/research/hubmap/tissue-bar-graphs/csv'
output_dir = 'C:/Users/catherine/Desktop/research/hubmap/ccf-cell-generation/hubmap_csv'
copy_csv_files_in_one_folder(input_dir, output_dir)

Step 2: for every tissue block, send a request and return the intersected volume and volume of AS.

In [16]:
file_mapping_rui_uuid_hubmap_id = './csv/mapping_rui_uuid_hubmap_id.csv'
file_mapping_rui_hubmap_id = './csv/mapping_rui_hubmap_id.csv'
hubmap_csv_folder = output_dir

mapping_rui_dataset_id = collections.defaultdict(list)

with open(file_mapping_rui_uuid_hubmap_id, 'r', encoding='UTF8') as f:
    csv_reader = csv.reader(f)
    for row in csv_reader:
        hubmap_id, uuid, rui = row
        mapping_rui_dataset_id[rui].append(hubmap_id)

with open(file_mapping_rui_hubmap_id, 'w', encoding='UTF8', newline="") as f:
    csv_writer = csv.writer(f)
    for rui, hubmap_ids in mapping_rui_dataset_id.items():
        csv_writer.writerow([rui] + hubmap_ids)

In [ ]:
with open(file_mapping_rui_hubmap_id, 'r', encoding='UTF8') as f:
    csv_reader = csv.reader(f)
    for row in csv_reader:
        rui_url = row[0]
        hubmap_ids = row[1:]

        for hubmap_id in hubmap_ids:
            csv_path = os.path.join(hubmap_csv_folder, hubmap_id + '.csv')
            dic_cell_type_count = collections.defaultdict(list)
            dic_cell_type_avg_count = collections.Counter()
            with open(csv_path, 'r', encoding='UTF8') as csv_file:
                dataset_csv_reader = csv.reader(csv_file)
                header = next(dataset_csv_reader)
                cell_type_index = header.index('cell_type')
                count_index = header.index('count')

                for item_cell in dataset_csv_reader:
                    dic_cell_type_count[item_cell[cell_type_index]].append(item_cell[count_index])

                for cell, count_list in dic_cell_type_count.items():
                    dic_cell_type_avg_count[cell] = sum(count_list) / len(count_list)


        get_response = requests.get(rui_url).json()
        rui_location = get_response['rui_location']
        post_response = requests.post(url=url_collision_detection, json=rui_location)

        mapping_AS_cell_type_count = collections.defaultdict(dict)

        try:
            post_response = post_response.json()
            for AS_item in post_response:
                percentage_of_AS = AS_item['percentage_of_AS']
                organ = AS_item['organ']
                node_name = AS_item['node_name']
                for cell, avg_count in dic_cell_type_avg_count.items():
                    mapping_AS_cell_type_count[(organ, node_name)][cell] = avg_count / percentage_of_AS

        except:
            pass
